Show how to use pretrained assertion status

In [2]:
from google.colab import drive
drive.mount("/content/gdrive/")
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/SparkNLP/utils')
!ls

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
gdrive	sample_data


In [4]:
drivepath = "gdrive/My Drive/Colab Notebooks/SparkNLP/utils"
biotrainset = "con_rest_train.bio"
gloveemb = "glove.6B.300d.txt"
modelsavefilename="ner_dl_model_aug"
print("Driver path file name :"+drivepath)
print("BIO file name :"+biotrainset)
print("Glovec file name :"+gloveemb)
print("modelsavefilename file name :"+modelsavefilename)
!cp "gdrive/My Drive/Colab Notebooks/SparkNLP/utils/con_rest_train.bio" "con_rest_train.bio"
!cp "gdrive/My Drive/Colab Notebooks/SparkNLP/utils/glove.6B.300d.txt" "glove.6B.300d.txt"
#!cp "gdrive/My Drive/Colab Notebooks/SparkNLP/utils/blstm_17_300_128_37.pb" "blstm_17_300_128_37.pb"
!cp "gdrive/My Drive/Colab Notebooks/SparkNLP/utils/blstm-noncontrib_17_300_128_75.pb" "blstm-noncontrib_17_300_128_75.pb"


Driver path file name :gdrive/My Drive/Colab Notebooks/SparkNLP/utils
BIO file name :con_rest_train.bio
Glovec file name :glove.6B.300d.txt
modelsavefilename file name :ner_dl_model_aug


In [5]:
!ls

blstm_29_300_128_81.pb	gdrive		   sample_data	untitled
con_rest_train.bio	glove.6B.300d.txt  tf


In [6]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.3

# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.2.2

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)
     |████████████████████████████████| 215.6MB 49kB/s 
     |████████████████████████████████| 204kB 37.5MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.3-py2.py3-none-any.whl size=215964963 sha256=333b6150dd66faec988accbff94eabaf742d51d2d4b6dd5cc4a0f7483d9ef403
  Stored in directory: /root/.cache/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark


In [0]:
import sys

from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel

import sparknlp
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.pretrained import ResourceDownloader

from pathlib import Path

if sys.version_info[0] < 3:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve    

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Global DEMO - Spark NLP Enterprise 2.3.4") \
    .master("local[*]") \
    .config("spark.rdd.compress","true") \
    .config("spark.driver.memory","8G") \
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "600M") \
    .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:2.3.4") \
    .getOrCreate()
#spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)


2.2.2
Spark NLP version:  None
Apache Spark version:  2.4.3


In [9]:
!ls
from sparknlp.training import CoNLL
conll = CoNLL(
    documentCol="document",
    sentenceCol="sentence",
    tokenCol="token",
    posCol="pos",
    conllLabelIndex=3,
    conllPosIndex=1,
    textCol='text',
    labelCol='label',
    explodeSentences=True
)
training_data = conll.readDataset(spark, 'con_rest_train.bio',ReadAs.LINE_BY_LINE)
#training_data = CoNLL().readDataset(spark, 'con_rest_train.bio')
training_data,test_data=training_data.randomSplit([0.9, 0.1], None)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))
training_data.show(n=20)

blstm_29_300_128_81.pb	gdrive		   sample_data	untitled
con_rest_train.bio	glove.6B.300d.txt  tf
Training Dataset Count: 10963
Test Dataset Count: 1140
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|At meal time whil...|[[document, 0, 43...|[[document, 0, 43...|[[token, 0, 1, At...|[[pos, 0, 1, , [w...|[[named_entity, 0...|
|At thirteen hundr...|[[document, 0, 60...|[[document, 0, 60...|[[token, 0, 1, At...|[[pos, 0, 1, , [w...|[[named_entity, 0...|
|Can you give me t...|[[document, 0, 39...|[[document, 0, 39...|[[token, 0, 2, Ca...|[[pos, 0, 2, , [w...|[[named_entity, 0...|
|Can you give me t...|[[document, 0, 66...|[[document, 0, 66...|[[token, 0, 2, Ca

Create a custom pipeline

In [0]:
import time

documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddings = WordEmbeddings() \
        .setInputCols(["document", "token"])\
        .setOutputCol("embeddings")\
        .setEmbeddingsSource("glove.6B.300d.txt", 300, "text")

#spell = NorvigSweetingModel.pretrained() \
#    .setInputCols(["token"]) \
#    .setOutputCol("spell")
#.setGraphFolder("gdrive/My Drive/Colab Notebooks/SparkNLP/utils/Graphs")\
ner_dl = NerDLApproach()\
    .setInputCols(["document", "token","embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(2)\
    .setRandomSeed(0)\
    .setVerbose(2)\
    .setBatchSize(9)\
    .setGraphFolder("./")\
    .setEnableOutputLogs(True)

converter = NerConverter()\
  .setInputCols(["document", "token", "ner"])\
  .setOutputCol("ner_converter")

finisher = Finisher() \
    .setInputCols(["ner", "ner_converter"]) \
    .setIncludeMetadata(True)

pipeline_fast_dl = Pipeline(stages = [
    documentAssembler, 
    tokenizer, 
    embeddings, 
    ner_dl,
    converter,
    finisher])

Now let's use these pipelines and see the results

In [11]:
%%time

start = time.time()
print("Start fitting")
prediction_model = pipeline_fast_dl.fit(training_data)
print("Fitting is ended")
print (time.time() - start)

Start fitting
Fitting is ended
597.9858376979828
CPU times: user 179 ms, sys: 34.4 ms, total: 213 ms
Wall time: 9min 57s


In [12]:
#prediction_data = spark.createDataFrame([["Maria is a nice place."],["any bbq places open before 5 nearby"]]).toDF("text")
#prediction_data.show()
prediction_model.transform(test_data).show(truncate=False)

+-------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                             |finished_ner                                                                                                  

In [17]:
prediction_data = spark.createDataFrame([["what is the whtr in bangalore karnataka"]]).toDF("text")
prediction_data.show(truncate=False)
prediction_model.transform(prediction_data).show(truncate=False)


+---------------------------------------+
|text                                   |
+---------------------------------------+
|what is the whtr in bangalore karnataka|
+---------------------------------------+

+---------------------------------------+----------------------------------------------------+----------------------------+-------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                   |finished_ner                                        |finished_ner_converter      |finished_ner_metadata                                                                                  |finished_ner_converter_metadata                                                                                                                  |
+---------------------------------------+

In [0]:
prediction_model.write().overwrite().save("ner_dl_model_aug")
!ls

In [0]:
!cp -r 'ner_dl_model_aug' 'gdrive/My Drive/Colab Notebooks/SparkNLP/utils/ner_dl_model_aug'

In [0]:
from pyspark.ml import PipelineModel, Pipeline

loaded_prediction_model = PipelineModel.read().load('ner_dl_model_aug')


In [0]:
prediction_data = spark.createDataFrame([["Maria is a nice place."],["any bbq places open before 5 nearby"],["2 star restaurants with inside dining"],['98 hong kong restaurant reasonable prices']]).toDF("text")
prediction_data.show()
loaded_prediction_model.transform(prediction_data).show(truncate=False)
prediction = loaded_prediction_model.transform(prediction_data)
prediction.select("finished_ner_metadata").show(truncate=False)
prediction.select("finished_ner").show(truncate=False)
prediction.select("finished_ner_converter_metadata").show(truncate=False)
prediction.select("finished_ner_converter").show(truncate=False)
#prediction.select("ner").show(truncate=False)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(prediction, {evaluator.metricName: "areaUnderROC"})))

In [0]:
from sparknlp.base import LightPipeline

lp = LightPipeline(loaded_prediction_model)
result = lp.annotate("Peter is a good person.")
for e in list(zip(result['token'], result['ner']))[:10]:
    print(e)

In [0]:
for stage in loaded_prediction_model.stages:
    print(stage)
print(loaded_prediction_model.stages[-1].stages)